In [1]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
from ResidualAttentionUnet import *
from metrics import *
from Datagens import *

from tensorflow import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers.experimental import preprocessing

np.set_printoptions(precision=3, suppress=True)

CSVLogger.on_test_begin = CSVLogger.on_train_begin
CSVLogger.on_test_batch_end = CSVLogger.on_epoch_end
CSVLogger.on_test_end = CSVLogger.on_train_end

In [2]:
model = build_resatt_unet(n_channels=3, ker_init='he_normal', dropout=0.2)
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing] )

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 16  2320        ['conv2d[0][0]']                 
                                )                                                             

                                                                                                  
 add_5 (Add)                    (None, 8, 8, 128)    0           ['conv2d_transpose[0][0]',       
                                                                  'conv2d_16[0][0]']              
                                                                                                  
 activation_1 (Activation)      (None, 8, 8, 128)    0           ['add_5[0][0]']                  
                                                                                                  
 conv2d_18 (Conv2D)             (None, 8, 8, 1)      129         ['activation_1[0][0]']           
                                                                                                  
 activation_2 (Activation)      (None, 8, 8, 1)      0           ['conv2d_18[0][0]']              
                                                                                                  
 up_sampli

 add_8 (Add)                    (None, 32, 32, 64)   0           ['conv2d_30[0][0]',              
                                                                  'dropout_6[0][0]']              
                                                                                                  
 conv2d_31 (Conv2D)             (None, 32, 32, 32)   2080        ['add_8[0][0]']                  
                                                                                                  
 activation_6 (Activation)      (None, 32, 32, 32)   0           ['conv2d_31[0][0]']              
                                                                                                  
 conv2d_33 (Conv2D)             (None, 32, 32, 32)   1056        ['activation_6[0][0]']           
                                                                                                  
 conv2d_transpose_2 (Conv2DTran  (None, 32, 32, 32)  9248        ['conv2d_33[0][0]']              
 spose)   

                                )                                                                 
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 128, 128, 16  64         ['conv2d_43[0][0]']              
 rmalization)                   )                                                                 
                                                                                                  
 concatenate_3 (Concatenate)    (None, 128, 128, 48  0           ['up_sampling2d_7[0][0]',        
                                )                                 'batch_normalization_3[0][0]']  
                                                                                                  
 conv2d_44 (Conv2D)             (None, 128, 128, 16  6928        ['concatenate_3[0][0]']          
                                )                                                                 
          

In [4]:
directory = [f.path for f in os.scandir(PATH) if f.is_dir()]


def generate_ids(dirLst):
    ids = []
    for i in range(0,len(dirLst)):
        ids.append(dirLst[i][dirLst[i].rfind('/')+1:])
    return ids

train_and_test_ids = generate_ids(directory); 

    
train_test_ids, val_ids = train_test_split(train_and_test_ids, train_size=236, test_size=60, random_state=14) 
train_ids, test_ids = train_test_split(train_test_ids, test_size=36, random_state=14) 

In [5]:
training_generator = DataGenerator2D(train_ids)
valid_generator = DataGenerator2D(val_ids)
test_generator = DataGenerator2D(test_ids)

In [6]:
csv_logger = CSVLogger('Logs/training/resattU_net_2d.log', separator=',', append=False)

eval_csv_logger = CSVLogger('Logs/testing/resattU_net_2d.log', separator=',', append=False)


callbacks = [
      keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1),
        csv_logger
    ]

In [7]:
K.clear_session()

history =  model.fit(training_generator,
                    epochs=100,
                    steps_per_epoch=len(train_ids),
                    callbacks= callbacks,
                    validation_data = valid_generator
                    )  
model.save("Models/2D/resattU_net_2d.h5")

Epoch 1/100
200/200 [==============================] - 68s 290ms/step - loss: 0.2092 - accuracy: 0.9769 - mean_io_u: 0.5242 - dice_coef: 0.3206 - dice_coef_necrotic: 0.1509 - dice_coef_edema: 0.2525 - dice_coef_enhancing: 0.1571 - val_loss: 0.0854 - val_accuracy: 0.9839 - val_mean_io_u: 0.3756 - val_dice_coef: 0.3015 - val_dice_coef_necrotic: 0.1788 - val_dice_coef_edema: 0.3072 - val_dice_coef_enhancing: 0.2153 - lr: 0.0010
Epoch 2/100
200/200 [==============================] - 56s 282ms/step - loss: 0.0406 - accuracy: 0.9882 - mean_io_u: 0.6688 - dice_coef: 0.4041 - dice_coef_necrotic: 0.2482 - dice_coef_edema: 0.4392 - dice_coef_enhancing: 0.3727 - val_loss: 0.0667 - val_accuracy: 0.9833 - val_mean_io_u: 0.6880 - val_dice_coef: 0.3611 - val_dice_coef_necrotic: 0.2249 - val_dice_coef_edema: 0.3100 - val_dice_coef_enhancing: 0.2334 - lr: 0.0010
Epoch 3/100
200/200 [==============================] - 56s 281ms/step - loss: 0.0337 - accuracy: 0.9899 - mean_io_u: 0.7486 - dice_coef: 0.452

Epoch 17/100
200/200 [==============================] - 55s 277ms/step - loss: 0.0236 - accuracy: 0.9925 - mean_io_u: 0.8009 - dice_coef: 0.5239 - dice_coef_necrotic: 0.3613 - dice_coef_edema: 0.6493 - dice_coef_enhancing: 0.6276 - val_loss: 0.1258 - val_accuracy: 0.9695 - val_mean_io_u: 0.8309 - val_dice_coef: 0.4202 - val_dice_coef_necrotic: 0.2841 - val_dice_coef_edema: 0.3860 - val_dice_coef_enhancing: 0.3522 - lr: 1.0000e-06
Epoch 18/100
200/200 [==============================] - 56s 278ms/step - loss: 0.0236 - accuracy: 0.9925 - mean_io_u: 0.8009 - dice_coef: 0.5238 - dice_coef_necrotic: 0.3613 - dice_coef_edema: 0.6486 - dice_coef_enhancing: 0.6280 - val_loss: 0.1262 - val_accuracy: 0.9694 - val_mean_io_u: 0.8313 - val_dice_coef: 0.4204 - val_dice_coef_necrotic: 0.2843 - val_dice_coef_edema: 0.3890 - val_dice_coef_enhancing: 0.3517 - lr: 1.0000e-06
Epoch 19/100
200/200 [==============================] - 55s 277ms/step - loss: 0.0236 - accuracy: 0.9925 - mean_io_u: 0.8010 - dice_

Epoch 36/100
200/200 [==============================] - 57s 285ms/step - loss: 0.0234 - accuracy: 0.9925 - mean_io_u: 0.8026 - dice_coef: 0.5253 - dice_coef_necrotic: 0.3629 - dice_coef_edema: 0.6518 - dice_coef_enhancing: 0.6292 - val_loss: 0.1249 - val_accuracy: 0.9698 - val_mean_io_u: 0.8316 - val_dice_coef: 0.4214 - val_dice_coef_necrotic: 0.2838 - val_dice_coef_edema: 0.3884 - val_dice_coef_enhancing: 0.3536 - lr: 1.0000e-06
Epoch 37/100
200/200 [==============================] - 64s 319ms/step - loss: 0.0234 - accuracy: 0.9925 - mean_io_u: 0.8028 - dice_coef: 0.5258 - dice_coef_necrotic: 0.3631 - dice_coef_edema: 0.6510 - dice_coef_enhancing: 0.6294 - val_loss: 0.1301 - val_accuracy: 0.9684 - val_mean_io_u: 0.8315 - val_dice_coef: 0.4203 - val_dice_coef_necrotic: 0.2847 - val_dice_coef_edema: 0.3887 - val_dice_coef_enhancing: 0.3537 - lr: 1.0000e-06
Epoch 38/100
200/200 [==============================] - 65s 327ms/step - loss: 0.0234 - accuracy: 0.9925 - mean_io_u: 0.8028 - dice_

Epoch 55/100
200/200 [==============================] - 57s 286ms/step - loss: 0.0232 - accuracy: 0.9926 - mean_io_u: 0.8049 - dice_coef: 0.5271 - dice_coef_necrotic: 0.3653 - dice_coef_edema: 0.6542 - dice_coef_enhancing: 0.6319 - val_loss: 0.1240 - val_accuracy: 0.9702 - val_mean_io_u: 0.8326 - val_dice_coef: 0.4232 - val_dice_coef_necrotic: 0.2834 - val_dice_coef_edema: 0.3893 - val_dice_coef_enhancing: 0.3555 - lr: 1.0000e-06
Epoch 56/100
200/200 [==============================] - 57s 287ms/step - loss: 0.0232 - accuracy: 0.9926 - mean_io_u: 0.8051 - dice_coef: 0.5274 - dice_coef_necrotic: 0.3645 - dice_coef_edema: 0.6542 - dice_coef_enhancing: 0.6325 - val_loss: 0.1299 - val_accuracy: 0.9683 - val_mean_io_u: 0.8318 - val_dice_coef: 0.4202 - val_dice_coef_necrotic: 0.2857 - val_dice_coef_edema: 0.3861 - val_dice_coef_enhancing: 0.3551 - lr: 1.0000e-06
Epoch 57/100
200/200 [==============================] - 58s 288ms/step - loss: 0.0232 - accuracy: 0.9926 - mean_io_u: 0.8051 - dice_

Epoch 74/100
200/200 [==============================] - 57s 287ms/step - loss: 0.0231 - accuracy: 0.9926 - mean_io_u: 0.8072 - dice_coef: 0.5288 - dice_coef_necrotic: 0.3666 - dice_coef_edema: 0.6572 - dice_coef_enhancing: 0.6344 - val_loss: 0.1247 - val_accuracy: 0.9699 - val_mean_io_u: 0.8334 - val_dice_coef: 0.4231 - val_dice_coef_necrotic: 0.2847 - val_dice_coef_edema: 0.3897 - val_dice_coef_enhancing: 0.3575 - lr: 1.0000e-06
Epoch 75/100
200/200 [==============================] - 58s 288ms/step - loss: 0.0230 - accuracy: 0.9926 - mean_io_u: 0.8073 - dice_coef: 0.5291 - dice_coef_necrotic: 0.3667 - dice_coef_edema: 0.6571 - dice_coef_enhancing: 0.6346 - val_loss: 0.1242 - val_accuracy: 0.9698 - val_mean_io_u: 0.8328 - val_dice_coef: 0.4224 - val_dice_coef_necrotic: 0.2856 - val_dice_coef_edema: 0.3880 - val_dice_coef_enhancing: 0.3561 - lr: 1.0000e-06
Epoch 76/100
200/200 [==============================] - 58s 288ms/step - loss: 0.0230 - accuracy: 0.9926 - mean_io_u: 0.8075 - dice_

Epoch 93/100
200/200 [==============================] - 57s 286ms/step - loss: 0.0229 - accuracy: 0.9927 - mean_io_u: 0.8096 - dice_coef: 0.5308 - dice_coef_necrotic: 0.3689 - dice_coef_edema: 0.6600 - dice_coef_enhancing: 0.6367 - val_loss: 0.1325 - val_accuracy: 0.9675 - val_mean_io_u: 0.8334 - val_dice_coef: 0.4213 - val_dice_coef_necrotic: 0.2878 - val_dice_coef_edema: 0.3896 - val_dice_coef_enhancing: 0.3576 - lr: 1.0000e-06
Epoch 94/100
200/200 [==============================] - 58s 287ms/step - loss: 0.0229 - accuracy: 0.9927 - mean_io_u: 0.8097 - dice_coef: 0.5306 - dice_coef_necrotic: 0.3692 - dice_coef_edema: 0.6592 - dice_coef_enhancing: 0.6369 - val_loss: 0.1292 - val_accuracy: 0.9685 - val_mean_io_u: 0.8338 - val_dice_coef: 0.4230 - val_dice_coef_necrotic: 0.2858 - val_dice_coef_edema: 0.3938 - val_dice_coef_enhancing: 0.3591 - lr: 1.0000e-06
Epoch 95/100
200/200 [==============================] - 58s 287ms/step - loss: 0.0229 - accuracy: 0.9927 - mean_io_u: 0.8098 - dice_

In [ ]:
# model = keras.models.load_model('Models/U_net_200_2d_3ch.h5', 
#                                    custom_objects={ 'accuracy' : tf.keras.metrics.MeanIoU(num_classes=4),
#                                                    "dice_coef": dice_coef,
#                                                    "dice_coef_necrotic": dice_coef_necrotic,
#                                                    "dice_coef_edema": dice_coef_edema,
#                                                    "dice_coef_enhancing": dice_coef_enhancing
#                                                   }, compile=True)

In [ ]:
model.evaluate(test_generator, callbacks=eval_csv_logger, verbose=1)

In [ ]:
valid_csv_logger = CSVLogger('Logs/validation/resattU_net_2d.log', separator=',', append=False)
model.evaluate(valid_generator, callbacks=valid_csv_logger, verbose=1)